In [ ]:
import sys
from pathlib import Path

# Add src to sys.path for module imports
sys.path.append(str(Path.cwd().parent / "src"))

from database.DatabaseConnection import DatabaseConnection

# Connect to the database
db = DatabaseConnection(db_url="../slurm_data.db")

# Query jobs with GPUs
gpu_df = db.connection.query("SELECT * FROM Jobs WHERE GPUs > 0").to_df()

# Display the data
print(gpu_df.head())
print(f"Number of jobs with GPUs: {len(gpu_df)}")

In [ ]:
# Import analysis and preprocess functions
from analysis.vram_usage import EfficiencyAnalysis
from preprocess.preprocess import preprocess_data

new_df = preprocess_data(gpu_df, min_elapsed_second=0, include_failed_cancelled_jobs=False, include_CPU_only_job=True)

# Initialize the EfficiencyAnalysis class
efficiency_analyzer = EfficiencyAnalysis(df=new_df, table_name="Jobs")

# Load and preprocess data using the new class-based API
df_multi_gpu = efficiency_analyzer.jobs_df

# Display the loaded data
print(df_multi_gpu.head())

# Run the efficiency metrics calculation
efficiency_metrics = efficiency_analyzer.calculate_efficiency_metrics(
    gpus_min=1, vram_constraint_filter=0, gpu_mem_usage_min=0
)

# Display the calculated efficiency metrics
print(efficiency_metrics.head())
print(new_df.columns)

In [ ]:
# Calculate efficiency metrics
filtered_jobs = efficiency_analyzer.calculate_efficiency_metrics(
    vram_constraint_filter=0, allocated_vram_greater_than=0, gpu_mem_usage_min=0, gpus_min=1, elapsed_seconds_min=600
)

# Display the filtered jobs
filtered_jobs.head()

In [ ]:
import matplotlib.pyplot as plt

# results_df = db.connection.query("SELECT * FROM Jobs WHERE array_length(constraints, 1) > 1 and GPUs > 1").to_df()
# # results_df.to_csv("results.csv", index=False)
# results_df2 = db.connection.query("SELECT * FROM Jobs WHERE array_length(GPUType, 1) > 1").to_df()
# # results_df2.to_csv("results2.csv", index=False)

# Evaluate CPU-GPU usage
analysis_results = efficiency_analyzer.evaluate_cpu_gpu_usage(
    hours_percentage_threshold=25, vram_efficiency_threshold=0.3
)

# Display key summary statistics
print("Total jobs:", analysis_results["total_jobs"])
print("Total GPU hours:", f"{analysis_results['total_gpu_hours']:.2f}")
print("Average VRAM efficiency:", f"{analysis_results['avg_efficiency']:.2%}")
print("Median VRAM efficiency:", f"{analysis_results['median_efficiency']:.2%}")

# Show recommendations
print("\nRecommendations:")
for rec in analysis_results["report"]:
    print("-", rec)

# Display efficiency patterns table
display(analysis_results["efficiency_patterns"])

# Visualize the analysis results
cpu_gpu_balance = analysis_results.get("cpu_gpu_balance", None)
(cpu_gpu_balance)
if cpu_gpu_balance is not None:
    plt.figure(figsize=(10, 6))
    # cpu_gpu_balance['Job_Count'].plot(kind='bar', color='green')
    plt.xlabel("Workload Type")
    plt.ylabel("Job Count")
    plt.title("CPU-GPU Balance Analysis")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

db.connection.query("SELECT * FROM Jobs WHERE array_length(GPUType, 1) > 1 AND GPUMemUsage > 0").to_df()

# Export the analysis results to a CSV file
# output_path = 'results/main_db_analysis.csv'
# zero_vram_analyzer.export_results_to_csv(analysis_results, output_path)
# print(f"Analysis results exported to {output_path}")

# Visualize efficiency patterns
efficiency_patterns = analysis_results["efficiency_patterns"]
plt.figure(figsize=(10, 6))
sns.barplot(x=efficiency_patterns.index, y=efficiency_patterns["GPU_Hours"])
plt.xlabel("Efficiency Category")
plt.ylabel("GPU Hours")
plt.title("Efficiency Patterns")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Apply additional filtering options
filtered_data = efficiency_analyzer.calculate_efficiency_metrics(
    allocated_vram_greater_than=0, gpus_min=1, gpu_mem_usage_min=0.1
)

# Display the filtered data
display(filtered_data.head())

# Identify inefficient users
inefficient_users = efficiency_analyzer.find_inefficient_users_weighted_by_hours(efficiency_threshold=0.3, min_jobs=5)

# Display the inefficient users
inefficient_users.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot VRAM efficiency distribution (x-axis cut at 1.0)
plt.figure(figsize=(8, 5))
sns.histplot(filtered_data["vram_efficiency"].dropna().clip(upper=1.0), bins=30, kde=True)
plt.xlabel("VRAM Efficiency (clipped at 1.0)")
plt.ylabel("Number of Jobs")
plt.title("Distribution of VRAM Efficiency")
plt.xlim(0, 1.0)
plt.show()

In [ ]:
# Plot with value labels for average efficiency
if inefficient_users is not None and not inefficient_users.empty:
    top_problematic = inefficient_users.head(10)
    plt.figure(figsize=(10, 5))
    ax = sns.barplot(y=top_problematic["User"], x=top_problematic["Avg_Weighted_VRAM_Efficiency"], orient="h")
    plt.xlabel("Average Weighted VRAM Efficiency")
    plt.ylabel("User")
    plt.title("Top 10 Problematic Users (Lowest Efficiency)")
    plt.xlim(0, 1.0)
    # Add value labels
   
    plt.show()
    display(top_problematic)
else:
    print("No inefficient user data available in results.")

In [ ]:
# group by "interactive" using number of hours
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

hybrid_jobs = filtered_data
hybrid_jobs["allocated_vram"] = np.where(
    (hybrid_jobs["gpu_memory_used_gb"] > 16) & (hybrid_jobs["GPUType"].iloc[0] in ["A100", "V100"]),
    32,
    hybrid_jobs["allocated_vram"],
)

# Group by job type and sum the hours
job_type_hours = hybrid_jobs.groupby("Interactive")["Elapsed"].sum().reset_index()

# Plot job type distribution
plt.figure(figsize=(8, 5))
ax = sns.barplot(x="Interactive", y="Elapsed", data=job_type_hours)

plt.xlabel("Job Type (Interactive vs Non-Interactive)")
plt.ylabel("Total Elapsed Seconds")
plt.title("Total Elapsed Seconds by Job Type (Interactive vs Non-Interactive)")
plt.xticks(rotation=45)
plt.tight_layout()
ax.invert_yaxis()  # This flips the y-axis so bars grow from bottom to top
plt.show()

In [ ]:
test = hybrid_jobs[hybrid_jobs["allocated_vram"] < hybrid_jobs["gpu_memory_used_gb"]]
print(test[["allocated_vram", "gpu_memory_used_gb", "GPUs", "vram_efficiency", "GPUType", "Status", "ExitCode"]])
display(test)

In [ ]:
results_df = db.connection.query("SELECT * FROM Jobs WHERE array_length(constraints, 1) > 1 and GPUs > 1").to_df()
# results_df.to_csv("results.csv", index=False)
results_df2 = db.connection.query("SELECT * FROM Jobs WHERE array_length(GPUType, 1) > 1").to_df()
# results_df2.to_csv("results2.csv", index=False)
results_df2

In [ ]:
db.connection.query("SELECT * FROM Jobs WHERE array_length(GPUType, 1) > 1 AND GPUMemUsage > 0").to_df()

In [ ]:
test2 = test[test["Status"] != "TIMEOUT"]
test3 = test2[~test2["JobName"].isin(["bash", "interactive"])]
test3.rename(columns={"gpu_memory_used_gb": "GPUMemoryUsed"}, inplace=True)
test3[
    [
        "JobID",
        "allocated_vram",
        "Constraints",
        "GPUMemoryUsed",
        "vram_efficiency",
        "GPUType",
        "Status",
        "GPUs",
        "Partition",
    ]
]
# df[df["JobID"] == 24181142]

db.connection.query("SELECT * FROM Jobs WHERE JobID = 28943041").to_df()

# Identify inefficient PIs
inefficient_pis = efficiency_analyzer.find_inefficient_pis_weighted_by_hours(efficiency_threshold=0.3, min_jobs=5)

# Display the inefficient PIs
display(inefficient_pis.head())

In [ ]:
# Print GPU count summary for hybrid jobs
print("Unique GPU counts in hybrid jobs:", hybrid_jobs["GPUs"].unique())
print("GPU count distribution in hybrid jobs:")
print(hybrid_jobs["GPUs"].value_counts())
print(
    f"Min GPUs: {hybrid_jobs['GPUs'].min()}, "
    f"Max GPUs: {hybrid_jobs['GPUs'].max()}, "
    f"Mean GPUs: {hybrid_jobs['GPUs'].mean():.2f}"
)

# Visualize inefficient users (top 10 only with nonzero values)
top_inefficient_users = inefficient_users[inefficient_users["Weighted_Efficiency_Contribution"] > 0].head(10)
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=top_inefficient_users["Weighted_Efficiency_Contribution"], y=top_inefficient_users["User"])
plt.xlabel("Weighted Efficiency Contribution")
plt.ylabel("User")
plt.title("Top 10 Inefficient Users")

# # Add value labels to the bars
# for index, row in top_inefficient_users.iterrows():
#     ax.text(row["Weighted_Efficiency_Contribution"] + 0.01, index, f"{row['Weighted_Efficiency_Contribution']:.2f}", va="center", fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Plot with value labels for average efficiency
if "top_inefficient_users" in analysis_results:
    top_problematic = analysis_results["top_inefficient_users"].head(10)
    plt.figure(figsize=(10, 5))
    ax = sns.barplot(y=top_problematic.index, x=top_problematic["Avg_Efficiency"], orient="h")
    plt.xlabel("Average VRAM Efficiency")
    plt.ylabel("User")
    plt.title("Top 10 Problematic Users (Lowest Efficiency)")
    plt.xlim(0, 1.0)
    # Add value labels
    for _, (eff, y) in enumerate(zip(top_problematic["Avg_Efficiency"], ax.get_yticks(), strict=False)):
        ax.text(eff + 0.01, y, f"{eff:.2%}", va="center", fontsize=9)
    plt.show()
    display(top_problematic)
else:
    print("No inefficient user data available in results.")

In [ ]:
# Extract the first element of GPUType array and create a new field
hybrid_jobs["GPUType_First"] = hybrid_jobs["GPUType"].apply(lambda x: x[0])
print(hybrid_jobs.columns)
# Group by GPUType_First and calculate statistics
gpu_stats_first = hybrid_jobs.groupby("GPUType_First").agg(
    {
        "vram_efficiency": ["mean", "std"],
        "gpu_memory_used_gb": ["mean", "std"],
        "allocated_vram": ["mean", "std"],
        "gpu_hours": ["mean", "std"],
    }
)
print("Statistics grouped by GPUType_First:")
print(gpu_stats_first)

# Plot most requested GPUs (first element)
plt.figure(figsize=(12, 6))
requested_gpu_counts_first = hybrid_jobs["GPUType_First"].value_counts()
sns.barplot(x=requested_gpu_counts_first.index, y=requested_gpu_counts_first.values)
plt.title("Most Assigned GPUs (First Element)")
plt.xlabel("GPUType_First")
plt.ylabel("Request Count")
plt.xticks(rotation=45)
plt.show()

In [ ]:
hybrid_jobs[hybrid_jobs["vram_efficiency"] > 1][
    [
        "JobID",
        "Interactive",
        "Status",
        "ExitCode",
        "Constraints",
        "GPUs",
        "GPUMemUsage",
        "gpu_memory_used_gb",
        "allocated_vram",
        "GPUType",
        "vram_efficiency",
    ]
]

In [ ]:
# Filter multi-GPU jobs
import numpy as np

hybrid_jobs["allocated_vram"] = np.where(
    (hybrid_jobs["gpu_memory_used_gb"] > 16) & (hybrid_jobs["GPUType"].iloc[0][0] == "a100",),
    32,
    hybrid_jobs["allocated_vram"],
)
hybrid_jobs["allocated_vram"] = np.where(
    (hybrid_jobs["gpu_memory_used_gb"] > 16) & (hybrid_jobs["GPUType"].iloc[0][0] == "v100"),
    32,
    hybrid_jobs["allocated_vram"],
)

multi_gpu_jobs = hybrid_jobs[(hybrid_jobs["GPUs"] > 0) & (~hybrid_jobs["Status"].isin(["TIMEOUT", "OUT_OF_MEMORY"]))]
print(f"Number of multi-GPU jobs: {len(multi_gpu_jobs)}")

# Summarize multi-GPU job efficiency
multi_gpu_summary = multi_gpu_jobs.groupby("GPUs").agg(
    {"vram_efficiency": ["mean", "std"], "gpu_hours": ["mean", "std"], "GPUMemUsage": ["mean", "std"]}
)
print("Multi-GPU job summary:")
print(multi_gpu_summary)

# Visualize efficiency distribution for multi-GPU jobs
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.boxplot(x="GPUs", y="vram_efficiency", data=multi_gpu_jobs)
plt.title("Efficiency Distribution by GPU Count")
plt.xlabel("GPU Count")
plt.ylabel("Efficiency")
plt.show()
multi_gpu_jobs[multi_gpu_jobs["vram_efficiency"] > 1][
    [
        "JobID",
        "Status",
        "ExitCode",
        "Constraints",
        "GPUs",
        "gpu_memory_used_gb",
        "allocated_vram",
        "GPUType",
        "vram_efficiency",
    ]
].sort_values(by="vram_efficiency", ascending=False)

In [ ]:
# Plot GPU Utilization vs. GPU Count
plt.figure(figsize=(10, 6))
sns.scatterplot(x="GPUs", y="GPUMemUsage", data=multi_gpu_jobs)
plt.title("GPU Utilization vs. GPU Count")
plt.xlabel("GPU Count")
plt.ylabel("GPU Memory Usage")
plt.show()

In [ ]:
# Plot Job Duration vs. GPU Count
plt.figure(figsize=(10, 6))
sns.boxplot(x="GPUs", y="gpu_hours", data=multi_gpu_jobs)
plt.title("Job Duration Distribution by GPU Count")
plt.xlabel("GPU Count")
plt.ylabel("Job Duration (hours)")
plt.show()

In [ ]:
# Plot VRAM Efficiency Over Time
plt.figure(figsize=(10, 6))
sns.lineplot(x="Elapsed", y="vram_efficiency", data=multi_gpu_jobs.sort_values("Elapsed"))
plt.title("VRAM Efficiency Over Time")
plt.xlabel("Elapsed Time")
plt.ylabel("VRAM Efficiency")
plt.xticks(rotation=45)
plt.show()